In [1]:
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
from gliner import GLiNER

In [2]:
task_id = "31"

thr = 0.65

PATH = "models/checkpoint-1840" # "urchade/gliner_small-v2.1"

model = GLiNER.from_pretrained(PATH,).cuda() #  load_tokenizer=True

config.json not found in /data/home/eak/learning/zindi_challenge/micro_rec/models/checkpoint-1840


In [3]:
import re

def split_and_clean(sentence: str):
	# Split sentence into words while keeping punctuation as separate tokens
	words = re.findall(r"[\w]+|[^\s\w]", sentence)
	return words

def normalize_word(word: str):
	# Remove leading # and @ symbols for comparison purposes
	return word.lstrip("#@")

In [4]:
test_data = pd.read_csv("microsoft-learn-location-mention-recognition-challenge20240905-10153-193u9hv/Test.csv")

test_data = test_data.fillna("")

# test_data["text"] = test_data["text"].apply(lambda x: " ".join(split_and_clean(x)))
test_data.sample(10)

,tweet_id,text
1225,ID_1167962033284878336,RT @baltimoresun: Maryland rescue crews head t...
1658,ID_728738932909875200,Northwestern Ontario Wildfire Update – May 6 2...
1122,ID_1112831621017763840,Today @SecWilkie held a call with Nebraska @Go...
1346,ID_1176479940600053760,5.8 Magnitude earthquake hits #Pakistan-admini...
1516,ID_721802440253812736,Ecuador needs help right now. #EcuadorEarthqua...
794,ID_1106941332558217216,"An estimated 600,000 people in #Mozambique hav..."
1314,ID_1168376879708901376,For those of you in NORTH CAROLINA and surroun...
361,ID_1032923987913986048,#KeralaFloods have caused destruction but have...
2316,ID_902891297249353728,Two killed Tuesday night when tree falls on pi...
2888,ID_914154121061691392,὎2 #PuertoRico Relief efforts in Puerto Rico a...


In [5]:
from tqdm import tqdm

texts = test_data["text"].tolist()
bsize = 256

predictions = [
	model.batch_predict_entities(texts[i: i+bsize], ["disaster related location"], threshold=.05) for i in tqdm(range(0, len(texts), bsize))
]

len(predictions)

  0%|          | 0/12 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 12/12 [00:14<00:00,  1.23s/it]


12

In [11]:
all_predictions = sum(predictions, start=[])

def drop_duplicate(places):
    return sorted(list(set(places)))
# 0.65
raws = [
	" ".join(drop_duplicate(i["text"] for i in pred if i["score"] > .7)) or " " for pred in all_predictions
]

In [12]:
submission = test_data[["tweet_id"]]
submission["location"] = raws

submission.sample(10)

,tweet_id,location
458,ID_1039546242542522368,Florida
1628,ID_722204384139087872,Kumamoto
423,ID_1037567855112474624,India Kerala Uttarakhand
2773,ID_912363705521905664,Mexicos
1554,ID_721947100741836800,Ecuador India
1235,ID_1168178071679225856,Cape Canaveral FL
1589,ID_722089934144872448,Ecuador
1005,ID_1111005776255033344,Nebraska Ohio United States
2645,ID_910738575938822144,Mexicos
2577,ID_910624637033357312,Mexico


In [13]:
submission[submission["location"] == " "]

,tweet_id,location
76,ID_1021722786316935168,
106,ID_1021799419681878016,
151,ID_1022166211927449600,
170,ID_1022461309324079104,
260,ID_1030761481527275520,
...,...,...
2824,ID_913186608794492928,
2853,ID_913458667625881600,
2891,ID_914190210728173568,
2895,ID_914205749013893120,


In [14]:
submission.to_csv(f"submissions/{task_id}-GlinerLargeSplit.csv", index=False)

In [15]:
f"submissions/{task_id}-GlinerLargeSplit.csv"

'submissions/31-GlinerLargeSplit.csv'

: 